## Employment rates in Baltimore NIIF areas: 2017-2021 ACS 5-Year estimates

In [1]:
# From https://www.census.gov/data/developers/data-sets/acs-5year.html
# ACS has non-overlapping datasets that allow comparisons of current ACS data to past ACS data.  
# The 2017-2021 ACS 5-Year estimates can be compared with 2012-2016 ACS 5-Year estimates. 

In [2]:
# see https://towardsdatascience.com/using-the-us-census-api-for-data-analysis-a-beginners-guide-98063791785c
# For API call example, see https://api.census.gov/data/2021/acs/acs5/examples.html

In [3]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

In [4]:
pd.set_option('display.max_rows', 200)
pd.set_option('max_colwidth', 100)

In [5]:
# GIS Analyst, Patrick, provided a csv (created from ARCGIS) that contains a key to match Baltimore's NIIF areas with census tracts

# Store filepath in a variable
NIIFs = "./raw_data/NIIF_Tracts.csv"

# Read the file with the pandas library
# make sure all of these NIIF census tracts are pulled from the census API (see below)
NIIFs_df = pd.read_csv(NIIFs)
NIIFs_df

,GEOID,NAMELSAD
0,24510272007,Census Tract 2720.07
1,24510280500,Census Tract 2805
2,24510260202,Census Tract 2602.02
3,24510260203,Census Tract 2602.03
4,24510260301,Census Tract 2603.01
5,24510260302,Census Tract 2603.02
6,24510260303,Census Tract 2603.03
7,24510260401,Census Tract 2604.01
8,24510090200,Census Tract 902
9,24510090300,Census Tract 903


In [6]:
# Performing a GET Request and saving the API's response within a variable ("url" is the variable)
# if you get errors, the year you're querying may not be available (response 404 you will get) or there may be a space in your code that you need to remove
# used "\ at end of each line of code and " at beginning of each line so that could use multiple lines of code without causing errors

# Note that all of the census tracts included below were copied from the NIIF census tract csv file provided by Patrick/ GIS Analyst

url = "https://api.census.gov/data/2021/acs/acs5?get=NAME,B23025_001E,B23025_002E,B23025_004E,B23025_005E"\
"&for=tract:260403,272007,280500,260202,260203,260301,"\
"260303,260401,090200,090300,090400,090500,090600,090700,260403,260402,"\
"260404,260501,090800,100200,120300,120500,280600,260605,270101,270102,"\
"270200,270301,270302,210100,210200,271600,271700,270600,270401,151200,"\
"151300,160400,160700,160801,160802,270702,270801,270703,270802,130100,"\
"170200,170300,200200,200300,200400,270803,270805,270901,270902,270804,"\
"180300,190100,190200,200500,200600,200701,200800,270903,271001,271002,270402,"\
"270502,90200,190300,200100,250101,250102,250103,250203,250204,060300,"\
"080800,270501,160200,100100,130300,060100,280301,160100,130200,130400,"\
"250205,150100,090900,120400,270701,080301,271801,150900,080700,261000,"\
"160300,260604,280402,160500,130806,140200,140300,150200,150300,150400,"\
"150600,150701,150702,271802,060200,200702,160600,151000,151100,150800,"\
"250206,250207,250301,250303,250401,250402,280101,280102,280200,280302,"\
"060400,070100,070200,070300,070400,260101,260102,260201,280401,280403,"\
"280404,080101,080102,080200,080302,080400,080500,080600,090100,250500,"\
"250600&in=state:24%20county:510&key=073b804c85c7e45d3710695f0dde8b3c721fdecd"
response = requests.get(url)

response.status_code
#response_json = response.json()
#print(json.dumps(response_json, indent=4, sort_keys=True))

200

In [7]:
#  Get response from census in json format and convert to pandas dataframe
census = response.text
census = json.loads(census)
df = pd.DataFrame.from_dict(census)
df

,0,1,2,3,4,5,6,7
0,NAME,B23025_001E,B23025_002E,B23025_004E,B23025_005E,state,county,tract
1,"Census Tract 601, Baltimore city, Maryland",2218,1608,1519,89,24,510,060100
2,"Census Tract 602, Baltimore city, Maryland",2205,1537,1308,221,24,510,060200
3,"Census Tract 603, Baltimore city, Maryland",1363,1054,975,59,24,510,060300
4,"Census Tract 604, Baltimore city, Maryland",1567,984,878,106,24,510,060400
5,"Census Tract 701, Baltimore city, Maryland",1833,826,707,87,24,510,070100
6,"Census Tract 702, Baltimore city, Maryland",2333,1166,960,206,24,510,070200
7,"Census Tract 703, Baltimore city, Maryland",920,553,484,69,24,510,070300
8,"Census Tract 704, Baltimore city, Maryland",1357,752,563,189,24,510,070400
9,"Census Tract 801.01, Baltimore city, Maryland",2889,1954,1881,55,24,510,080101


In [8]:
# use the code below to change the first row of the dataframe into column headers
df.columns = df.iloc[0]
df = df.iloc[1:].reset_index(drop=True)
df

,NAME,B23025_001E,B23025_002E,B23025_004E,B23025_005E,state,county,tract
0,"Census Tract 601, Baltimore city, Maryland",2218,1608,1519,89,24,510,060100
1,"Census Tract 602, Baltimore city, Maryland",2205,1537,1308,221,24,510,060200
2,"Census Tract 603, Baltimore city, Maryland",1363,1054,975,59,24,510,060300
3,"Census Tract 604, Baltimore city, Maryland",1567,984,878,106,24,510,060400
4,"Census Tract 701, Baltimore city, Maryland",1833,826,707,87,24,510,070100
5,"Census Tract 702, Baltimore city, Maryland",2333,1166,960,206,24,510,070200
6,"Census Tract 703, Baltimore city, Maryland",920,553,484,69,24,510,070300
7,"Census Tract 704, Baltimore city, Maryland",1357,752,563,189,24,510,070400
8,"Census Tract 801.01, Baltimore city, Maryland",2889,1954,1881,55,24,510,080101
9,"Census Tract 801.02, Baltimore city, Maryland",1426,675,515,160,24,510,080102


In [9]:
import os

In [10]:
os.getcwd()

'C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\BT-equity-metric-analysis'

In [11]:
# if need to change the working directory; may need to change the directory for pulling in config file
#os.chdir('C:\\Users\\Jenkir\\Desktop\\BDC\\Baltimore_Together\\measuring-success-metrics\\BT-equity-metric-analysis')


In [12]:
# ACS Table codes to use in cell below;  Universe: Population 16 years and over 
# see https://www.census.gov/programs-surveys/acs/technical-documentation/table-shells.html
# Table B23025: Enployment Status
# B23025_002E	Estimate!!Total:!!In labor force:	B23025_002EA,B23025_002M,B23025_002MA
# B23025_003E	Estimate!!Total:!!In labor force:!!Civilian labor force
# B23025_004E	Estimate!!Total:!!In labor force:!!Civilian labor force:!!Employed	B23025_004EA,B23025_004M,B23025_004MA
# B23025_005E	Estimate!!Total:!!In labor force:!!Civilian labor force:!!Unemployed	B23025_005EA,B23025_005M,B23025_005MA


In [13]:
# rename columns
ACS5_2021 = df.rename(columns={"B23025_001E": "Population age 16 and over",
                               "B23025_002E": "In labor force",
                               "B23025_004E": "# employed, age 16 and over",
                               "B23025_005E": "# unemployed age 16 and over",
                               "NAME": "Name"})
ACS5_2021

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract
0,"Census Tract 601, Baltimore city, Maryland",2218,1608,1519,89,24,510,060100
1,"Census Tract 602, Baltimore city, Maryland",2205,1537,1308,221,24,510,060200
2,"Census Tract 603, Baltimore city, Maryland",1363,1054,975,59,24,510,060300
3,"Census Tract 604, Baltimore city, Maryland",1567,984,878,106,24,510,060400
4,"Census Tract 701, Baltimore city, Maryland",1833,826,707,87,24,510,070100
5,"Census Tract 702, Baltimore city, Maryland",2333,1166,960,206,24,510,070200
6,"Census Tract 703, Baltimore city, Maryland",920,553,484,69,24,510,070300
7,"Census Tract 704, Baltimore city, Maryland",1357,752,563,189,24,510,070400
8,"Census Tract 801.01, Baltimore city, Maryland",2889,1954,1881,55,24,510,080101
9,"Census Tract 801.02, Baltimore city, Maryland",1426,675,515,160,24,510,080102


In [14]:
ACS5_2021_final = ACS5_2021.copy()
ACS5_2021_final.head()

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract
0,"Census Tract 601, Baltimore city, Maryland",2218,1608,1519,89,24,510,060100
1,"Census Tract 602, Baltimore city, Maryland",2205,1537,1308,221,24,510,060200
2,"Census Tract 603, Baltimore city, Maryland",1363,1054,975,59,24,510,060300
3,"Census Tract 604, Baltimore city, Maryland",1567,984,878,106,24,510,060400
4,"Census Tract 701, Baltimore city, Maryland",1833,826,707,87,24,510,070100


In [15]:
# Add in employment rate (# employed, age 16 and over / Population age 16 and over)
ACS5_2021_final["Employment rate"] = 100 * \
    ACS5_2021_final["# employed, age 16 and over"].astype(
        int) / ACS5_2021_final["Population age 16 and over"].astype(int)
ACS5_2021_final

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract,Employment rate
0,"Census Tract 601, Baltimore city, Maryland",2218,1608,1519,89,24,510,060100,68.485122
1,"Census Tract 602, Baltimore city, Maryland",2205,1537,1308,221,24,510,060200,59.319728
2,"Census Tract 603, Baltimore city, Maryland",1363,1054,975,59,24,510,060300,71.533382
3,"Census Tract 604, Baltimore city, Maryland",1567,984,878,106,24,510,060400,56.030632
4,"Census Tract 701, Baltimore city, Maryland",1833,826,707,87,24,510,070100,38.570649
5,"Census Tract 702, Baltimore city, Maryland",2333,1166,960,206,24,510,070200,41.148736
6,"Census Tract 703, Baltimore city, Maryland",920,553,484,69,24,510,070300,52.608696
7,"Census Tract 704, Baltimore city, Maryland",1357,752,563,189,24,510,070400,41.488578
8,"Census Tract 801.01, Baltimore city, Maryland",2889,1954,1881,55,24,510,080101,65.109034
9,"Census Tract 801.02, Baltimore city, Maryland",1426,675,515,160,24,510,080102,36.115007


In [16]:
# Round rate to one decimal point
ACS5_2021_final["Employment rate"] = ACS5_2021_final["Employment rate"].astype(float).round(1)
ACS5_2021_final

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract,Employment rate
0,"Census Tract 601, Baltimore city, Maryland",2218,1608,1519,89,24,510,060100,68.5
1,"Census Tract 602, Baltimore city, Maryland",2205,1537,1308,221,24,510,060200,59.3
2,"Census Tract 603, Baltimore city, Maryland",1363,1054,975,59,24,510,060300,71.5
3,"Census Tract 604, Baltimore city, Maryland",1567,984,878,106,24,510,060400,56.0
4,"Census Tract 701, Baltimore city, Maryland",1833,826,707,87,24,510,070100,38.6
5,"Census Tract 702, Baltimore city, Maryland",2333,1166,960,206,24,510,070200,41.1
6,"Census Tract 703, Baltimore city, Maryland",920,553,484,69,24,510,070300,52.6
7,"Census Tract 704, Baltimore city, Maryland",1357,752,563,189,24,510,070400,41.5
8,"Census Tract 801.01, Baltimore city, Maryland",2889,1954,1881,55,24,510,080101,65.1
9,"Census Tract 801.02, Baltimore city, Maryland",1426,675,515,160,24,510,080102,36.1


In [17]:
# Add in labor force participation rate (In labor force / Population age 16 and over)
ACS5_2021_final["Labor force participation rate"] = 100 * \
    ACS5_2021_final["In labor force"].astype(
        int) / ACS5_2021_final["Population age 16 and over"].astype(int)
ACS5_2021_final

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract,Employment rate,Labor force participation rate
0,"Census Tract 601, Baltimore city, Maryland",2218,1608,1519,89,24,510,060100,68.5,72.497746
1,"Census Tract 602, Baltimore city, Maryland",2205,1537,1308,221,24,510,060200,59.3,69.705215
2,"Census Tract 603, Baltimore city, Maryland",1363,1054,975,59,24,510,060300,71.5,77.329420
3,"Census Tract 604, Baltimore city, Maryland",1567,984,878,106,24,510,060400,56.0,62.795150
4,"Census Tract 701, Baltimore city, Maryland",1833,826,707,87,24,510,070100,38.6,45.062739
5,"Census Tract 702, Baltimore city, Maryland",2333,1166,960,206,24,510,070200,41.1,49.978568
6,"Census Tract 703, Baltimore city, Maryland",920,553,484,69,24,510,070300,52.6,60.108696
7,"Census Tract 704, Baltimore city, Maryland",1357,752,563,189,24,510,070400,41.5,55.416360
8,"Census Tract 801.01, Baltimore city, Maryland",2889,1954,1881,55,24,510,080101,65.1,67.635860
9,"Census Tract 801.02, Baltimore city, Maryland",1426,675,515,160,24,510,080102,36.1,47.335203


In [18]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract code
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
ACS5_2021_final["GEOID"] = ACS5_2021_final['state'] + ACS5_2021_final['county'] + ACS5_2021_final['tract']
ACS5_2021_final

,Name,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,state,county,tract,Employment rate,Labor force participation rate,GEOID
0,"Census Tract 601, Baltimore city, Maryland",2218,1608,1519,89,24,510,060100,68.5,72.497746,24510060100
1,"Census Tract 602, Baltimore city, Maryland",2205,1537,1308,221,24,510,060200,59.3,69.705215,24510060200
2,"Census Tract 603, Baltimore city, Maryland",1363,1054,975,59,24,510,060300,71.5,77.329420,24510060300
3,"Census Tract 604, Baltimore city, Maryland",1567,984,878,106,24,510,060400,56.0,62.795150,24510060400
4,"Census Tract 701, Baltimore city, Maryland",1833,826,707,87,24,510,070100,38.6,45.062739,24510070100
5,"Census Tract 702, Baltimore city, Maryland",2333,1166,960,206,24,510,070200,41.1,49.978568,24510070200
6,"Census Tract 703, Baltimore city, Maryland",920,553,484,69,24,510,070300,52.6,60.108696,24510070300
7,"Census Tract 704, Baltimore city, Maryland",1357,752,563,189,24,510,070400,41.5,55.416360,24510070400
8,"Census Tract 801.01, Baltimore city, Maryland",2889,1954,1881,55,24,510,080101,65.1,67.635860,24510080101
9,"Census Tract 801.02, Baltimore city, Maryland",1426,675,515,160,24,510,080102,36.1,47.335203,24510080102


In [19]:
# Remove extraneous columns
ACS5_2021_final = ACS5_2021_final.drop(['Name','state','county','tract'], axis=1)
ACS5_2021_final

,Population age 16 and over,In labor force,"# employed, age 16 and over",# unemployed age 16 and over,Employment rate,Labor force participation rate,GEOID
0,2218,1608,1519,89,68.5,72.497746,24510060100
1,2205,1537,1308,221,59.3,69.705215,24510060200
2,1363,1054,975,59,71.5,77.329420,24510060300
3,1567,984,878,106,56.0,62.795150,24510060400
4,1833,826,707,87,38.6,45.062739,24510070100
5,2333,1166,960,206,41.1,49.978568,24510070200
6,920,553,484,69,52.6,60.108696,24510070300
7,1357,752,563,189,41.5,55.416360,24510070400
8,2889,1954,1881,55,65.1,67.635860,24510080101
9,1426,675,515,160,36.1,47.335203,24510080102


In [20]:
# change order of columns
ACS5_2021_final = ACS5_2021_final[['GEOID', 'Population age 16 and over', '# employed, age 16 and over', 
                                   '# unemployed age 16 and over',"Employment rate", 
                                   'In labor force',"Labor force participation rate"]]
ACS5_2021_final 

,GEOID,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,Employment rate,In labor force,Labor force participation rate
0,24510060100,2218,1519,89,68.5,1608,72.497746
1,24510060200,2205,1308,221,59.3,1537,69.705215
2,24510060300,1363,975,59,71.5,1054,77.329420
3,24510060400,1567,878,106,56.0,984,62.795150
4,24510070100,1833,707,87,38.6,826,45.062739
5,24510070200,2333,960,206,41.1,1166,49.978568
6,24510070300,920,484,69,52.6,553,60.108696
7,24510070400,1357,563,189,41.5,752,55.416360
8,24510080101,2889,1881,55,65.1,1954,67.635860
9,24510080102,1426,515,160,36.1,675,47.335203


In [21]:
# Prepare to create df where we will sum the population, # employed, # unemployed, in labor force for all of the NIIF tracts 
ACS5_2021_sum1 = ACS5_2021_final.copy()
ACS5_2021_sum1

,GEOID,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,Employment rate,In labor force,Labor force participation rate
0,24510060100,2218,1519,89,68.5,1608,72.497746
1,24510060200,2205,1308,221,59.3,1537,69.705215
2,24510060300,1363,975,59,71.5,1054,77.329420
3,24510060400,1567,878,106,56.0,984,62.795150
4,24510070100,1833,707,87,38.6,826,45.062739
5,24510070200,2333,960,206,41.1,1166,49.978568
6,24510070300,920,484,69,52.6,553,60.108696
7,24510070400,1357,563,189,41.5,752,55.416360
8,24510080101,2889,1881,55,65.1,1954,67.635860
9,24510080102,1426,515,160,36.1,675,47.335203


In [22]:
# Remove extraneous columns
ACS5_2021_sum = ACS5_2021_sum1.drop(['Employment rate','Labor force participation rate'], axis=1)
ACS5_2021_sum.dtypes

0
GEOID                           object
Population age 16 and over      object
# employed, age 16 and over     object
# unemployed age 16 and over    object
In labor force                  object
dtype: object

In [23]:
# change data types from "object"/ text to "float"/numeric
ACS5_2021_sum[['Population age 16 and over', '# employed, age 16 and over', '# unemployed age 16 and over',
               'In labor force']]= ACS5_2021_sum[['Population age 16 and over', '# employed, age 16 and over', '# unemployed age 16 and over',
               'In labor force']].astype(float)
ACS5_2021_sum.dtypes

0
GEOID                            object
Population age 16 and over      float64
# employed, age 16 and over     float64
# unemployed age 16 and over    float64
In labor force                  float64
dtype: object

In [24]:
#  sum the population age 16 and over and # employed , age 16 and over for all of the census tracts that are in NIIF boundary
# need to make sure the data types are numeric and not "object" first
# "s" is a Pandas series
s = ACS5_2021_sum[['Population age 16 and over', '# employed, age 16 and over', '# unemployed age 16 and over',
               'In labor force']].sum()

In [25]:
# Convert the series ("s") to a dataframe
sums2 = s.to_frame()
sums2

,0
0,
Population age 16 and over,341728.0
"# employed, age 16 and over",183906.0
# unemployed age 16 and over,18390.0
In labor force,202564.0


In [26]:
# "Flip" the df so we have column headers
sums = sums2.transpose()
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force
0,341728.0,183906.0,18390.0,202564.0


In [27]:
# Add in employment rate (# employed, age 16 and over / Population age 16 and over)
sums["Employment rate"] = 100 * \
    sums["# employed, age 16 and over"].astype(
        int) / sums["Population age 16 and over"].astype(int)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate
0,341728.0,183906.0,18390.0,202564.0,53.816486


In [28]:
# Round rate to one decimal point
sums["Employment rate"] = sums["Employment rate"].astype(float).round(1)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate
0,341728.0,183906.0,18390.0,202564.0,53.8


In [29]:
# Add in labor force participation rate (In labor force / Population age 16 and over)
sums["Labor force participation rate"] = 100 * \
    sums["In labor force"].astype(
        int) / sums["Population age 16 and over"].astype(int)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate,Labor force participation rate
0,341728.0,183906.0,18390.0,202564.0,53.8,59.276384


In [30]:
# Round rate to one decimal point
sums["Labor force participation rate"] = sums["Labor force participation rate"].astype(float).round(1)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate,Labor force participation rate
0,341728.0,183906.0,18390.0,202564.0,53.8,59.3


In [31]:
# Add in unemployment rate (# unemployed, age 16 and over / Population age 16 and over)
sums["Unemployment rate"] = 100 * \
    sums["# unemployed age 16 and over"].astype(
        int) / sums["Population age 16 and over"].astype(int)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate,Labor force participation rate,Unemployment rate
0,341728.0,183906.0,18390.0,202564.0,53.8,59.3,5.381473


In [32]:
# Round rate to one decimal point
sums["Unemployment rate"] = sums["Unemployment rate"].astype(float).round(1)
sums

,Population age 16 and over,"# employed, age 16 and over",# unemployed age 16 and over,In labor force,Employment rate,Labor force participation rate,Unemployment rate
0,341728.0,183906.0,18390.0,202564.0,53.8,59.3,5.4


In [33]:
# Export file as a CSV, without the Pandas index, but with the header
# Do not run this last code block until you have all previous code blocks in their final form:

sums.to_csv("Jupyter_nb_final_csvs/NIIF_employment_2017-2021.csv", index = False, header=True)